### torch-gpu 확인

In [1]:
import torch
print(torch.cuda.is_available())  # True면 CUDA GPU가 인식됨
print(torch.cuda.current_device())  # 현재 사용 중인 CUDA 디바이스
print(torch.cuda.get_device_name(0)) 

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Device name:", torch.cuda.get_device_name(device))
else:
    print("CUDA is not available. Using CPU.")

True
0
NVIDIA GeForce RTX 3070
CUDA is available. Device name: NVIDIA GeForce RTX 3070


### depth map live 시각화

In [2]:
import cv2
from transformers import pipeline
from PIL import Image
import numpy as np
import torch

# 깊이 추정 파이프라인 초기화
# "cuda"가 사용 가능하면 GPU를 사용, 그렇지 않으면 CPU를 사용
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)

    # 깊이 맵 추정
    depth_map = pipe(pil_image)['depth']
    depth_map = np.array(depth_map)

    # 포인트 클라우드 생성 (예시)
    # 여기에 포인트 클라우드 생성 코드를 추가하세요

    # 결과를 화면에 표시
    cv2.imshow('Depth Map', depth_map)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### 원본, depth map(rgb)

In [1]:
import cv2
from transformers import pipeline
from PIL import Image
import numpy as np
import torch
import open3d as o3d

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

def create_rgbd_image(rgb_image, depth_map):
    # 깊이 맵 크기를 맞추기 위해 RGB 이미지 크기 조정
    rgb_image = cv2.resize(rgb_image, (depth_map.shape[1], depth_map.shape[0]))
    
    # RGB와 깊이 이미지를 numpy 배열로 변환
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(rgb_image),
        o3d.geometry.Image(depth_map),
        convert_rgb_to_intensity=False
    )
    
    return rgbd_image

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)

    # 깊이 맵 추정
    depth_map = pipe(pil_image)['depth']
    depth_map = np.array(depth_map)

    # RGBD 이미지 생성
    rgbd_image = create_rgbd_image(rgb_frame, depth_map)
    
    # RGB 이미지와 깊이 맵을 OpenCV를 사용해 나란히 표시
    depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
    
    combined_image = np.hstack((cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR), depth_colormap))
    
    cv2.imshow('RGB and Depth Map', combined_image)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [5]:
import cv2
from transformers import pipeline
from PIL import Image
import numpy as np
import torch

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

def create_rgbd_image(rgb_image, depth_map):
    # 깊이 맵 크기를 맞추기 위해 RGB 이미지 크기 조정
    rgb_image = cv2.resize(rgb_image, (depth_map.shape[1], depth_map.shape[0]))
    
    # 깊이 맵을 3채널로 변환
    depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
    
    return depth_colormap

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)

    # 깊이 맵 추정
    depth_map = pipe(pil_image)['depth']
    depth_map = np.array(depth_map)
    
    # 흑백 깊이 맵 생성
    depth_map_gray = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # RGB 깊이 맵 생성
    depth_map_rgb = create_rgbd_image(rgb_frame, depth_map)
    
    # 원본, 흑백 깊이 맵, RGB 깊이 맵을 하나의 창에 나란히 배치
    rgb_frame_bgr = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
    combined_image = np.hstack((rgb_frame_bgr, cv2.cvtColor(depth_map_gray, cv2.COLOR_GRAY2BGR), depth_map_rgb))
    
    cv2.imshow('Camera | Grayscale Depth Map | RGB Depth Map', combined_image)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 포인트 클라우드

In [3]:
import cv2
from transformers import pipeline
from PIL import Image
import numpy as np
import torch
import open3d as o3d

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

def create_point_cloud(rgb_image, depth_map):
    # 깊이 맵 크기를 맞추기 위해 RGB 이미지 크기 조정
    rgb_image = cv2.resize(rgb_image, (depth_map.shape[1], depth_map.shape[0]))
    
    # RGB와 깊이 이미지를 numpy 배열로 변환
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(rgb_image),
        o3d.geometry.Image(depth_map),
        convert_rgb_to_intensity=False
    )
    
    # 카메라 매트릭스를 정의합니다 (여기서는 예시로 단순한 매트릭스를 사용)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    )
    
    # 포인트 클라우드를 생성합니다
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        intrinsic
    )
    
    # 포인트 클라우드를 원점에 맞추기 위해 좌표계를 변환합니다
    pcd.transform([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])
    
    return pcd

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)

    # 깊이 맵 추정
    depth_map = pipe(pil_image)['depth']
    depth_map = np.array(depth_map)

    # 포인트 클라우드 생성
    pcd = create_point_cloud(rgb_frame, depth_map)
    
    # 포인트 클라우드 시각화
    o3d.visualization.draw_geometries([pcd])

    # 결과를 화면에 표시
    cv2.imshow('RGB Frame', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 깊이에 따른 포인트 클라우드 색상 (Red-Violet)

In [ ]:
import cv2
from transformers import pipeline
from PIL import Image
import numpy as np
import torch
import open3d as o3d

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

def create_point_cloud(rgb_image, depth_map):
    # 깊이 맵 크기를 맞추기 위해 RGB 이미지 크기 조정
    rgb_image = cv2.resize(rgb_image, (depth_map.shape[1], depth_map.shape[0]))
    
    # 깊이 값을 정규화
    depth_min, depth_max = depth_map.min(), depth_map.max()
    depth_map_normalized = (depth_map - depth_min) / (depth_max - depth_min)
    
    # 깊이 값에 따라 색상을 매핑하는 함수
    def depth_to_color(value):
        colors = [
            (148, 0, 211),  # Violet
            (75, 0, 130),   # Indigo
            (0, 0, 255),    # Blue
            (0, 255, 0),    # Green
            (255, 255, 0),  # Yellow
            (255, 127, 0),  # Orange
            (255, 0, 0)     # Red
        ]
        n = len(colors)
        value_scaled = value * (n - 1)
        idx1 = int(value_scaled)
        idx2 = min(idx1 + 1, n - 1)
        alpha = value_scaled - idx1
        return tuple([int(colors[idx1][i] * (1 - alpha) + colors[idx2][i] * alpha) for i in range(3)])

    # 깊이 값을 색상으로 변환
    color_map = np.zeros((depth_map.shape[0], depth_map.shape[1], 3), dtype=np.uint8)
    for i in range(depth_map.shape[0]):
        for j in range(depth_map.shape[1]):
            color_map[i, j] = depth_to_color(depth_map_normalized[i, j])
    
    # RGB와 깊이 이미지를 numpy 배열로 변환
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(color_map),
        o3d.geometry.Image(depth_map),
        convert_rgb_to_intensity=False
    )
    
    # 카메라 매트릭스를 정의합니다 (여기서는 예시로 단순한 매트릭스를 사용)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    )
    
    # 포인트 클라우드를 생성합니다
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        intrinsic
    )
    
    # 포인트 클라우드를 원점에 맞추기 위해 좌표계를 변환합니다
    pcd.transform([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])
    
    return pcd

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # OpenCV 프레임을 PIL 이미지로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)

    # 깊이 맵 추정
    depth_map = pipe(pil_image)['depth']
    depth_map = np.array(depth_map)

    # 포인트 클라우드 생성
    pcd = create_point_cloud(rgb_frame, depth_map)
    
    # 포인트 클라우드 시각화
    o3d.visualization.draw_geometries([pcd])

    # 결과를 화면에 표시
    cv2.imshow('RGB Frame', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 객체 인식

### 깊이 맵을 이용한 객체 인식

In [2]:
import os
import cv2
import requests
from transformers import pipeline
from PIL import Image
import numpy as np
import torch

# 파일 다운로드 함수
def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# 파일 다운로드
if not os.path.exists("yolov3.weights"):
    download_file("https://pjreddie.com/media/files/yolov3.weights", "yolov3.weights")

if not os.path.exists("yolov3.cfg"):
    download_file("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", "yolov3.cfg")

if not os.path.exists("coco.names"):
    download_file("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", "coco.names")


In [5]:
print("Frame slice shape:", frame[y:y+h, x:x+w].shape)
print("Depth colormap shape:", depth_colormap.shape)

Frame slice shape: (219, 276, 3)
Depth colormap shape: (219, 282, 3)


### 개선된 객체인식

### 깊이 맵 유무 객체인식 (상대거리)

In [1]:
import cv2
import numpy as np
from transformers import pipeline
from PIL import Image
import torch

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("depth-estimation", device=device)

# 객체 인식 모델 초기화 (YOLOv3 모델 사용)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# COCO 클래스 이름 불러오기
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# 웹캠을 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # 객체 인식 (깊이 맵 사용하지 않음)
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    frame_no_depth = frame.copy()
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame_no_depth, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame_no_depth, label + " " + str(round(confidence, 2)), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)

    # 객체 인식 (깊이 맵 사용)
    frame_with_depth = frame.copy()
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame_with_depth, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame_with_depth, label + " " + str(round(confidence, 2)), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
            
            # 객체 영역 추출 및 깊이 맵 추정
            object_rgb = frame[y:y+h, x:x+w]
            object_rgb = cv2.cvtColor(object_rgb, cv2.COLOR_BGR2RGB)
            pil_object_image = Image.fromarray(object_rgb)

            # 객체 영역의 깊이 맵 추정
            depth_result = pipe(pil_object_image)
            depth_map = np.array(depth_result['depth'])

            # 깊이 맵 시각화
            depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
            depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
            depth_colormap = cv2.resize(depth_colormap, (w, h))

            # BGR로 변환
            depth_colormap = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2BGR)

            # 크기 및 채널 일치 확인
            if frame_with_depth[y:y+h, x:x+w].shape == depth_colormap.shape:
                # 깊이 맵을 원본 프레임에 합성
                frame_with_depth[y:y+h, x:x+w] = cv2.addWeighted(frame_with_depth[y:y+h, x:x+w], 0.7, depth_colormap, 0.3, 0)

                # 객체의 중심 깊이 값 계산
                center_depth = depth_map[depth_map.shape[0] // 2, depth_map.shape[1] // 2]
                distance = center_depth * 1000  # 깊이 값을 mm로 변환 (예제용으로 1000을 곱함)
                cv2.putText(frame_with_depth, f'Distance: {distance:.2f} mm', (x, y - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
            else:
                print(f"Shapes do not match! Frame slice shape: {frame_with_depth[y:y+h, x:x+w].shape}, Depth colormap shape: {depth_colormap.shape}")

    # 좌우 화면 결합
    combined_frame = np.hstack((frame_no_depth, frame_with_depth))

    # 결과를 화면에 표시
    cv2.imshow('Object Detection: No Depth Map (Left) | With Depth Map (Right)', combined_frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Shapes do not match! Frame slice shape: (235, 360, 3), Depth colormap shape: (236, 360, 3)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Shapes do not match! Frame slice shape: (243, 523, 3), Depth colormap shape: (246, 523, 3)
Shapes do not match! Frame slice shape: (251, 509, 3), Depth colormap shape: (252, 509, 3)
Shapes do not match! Frame slice shape: (256, 514, 3), Depth colormap shape: (258, 514, 3)
Shapes do not match! Frame slice shape: (254, 544, 3), Depth colormap shape: (256, 544, 3)
Shapes do not match! Frame slice shape: (254, 576, 3), Depth colormap shape: (258, 576, 3)
Shapes do not match! Frame slice shape: (234, 355, 3), Depth colormap shape: (235, 355, 3)


# 손 제스처 인식

### 손의 상대거리

In [6]:
import cv2
import numpy as np
from transformers import pipeline
from PIL import Image
import torch
import mediapipe as mp

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
depth_pipe = pipeline("depth-estimation", device=device)

# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 손 랜드마크 검출
    results = hands.process(rgb_frame)

    depth_frame = frame.copy()  # 깊이 정보 포함한 프레임 복사본

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 랜드마크 그리기
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(depth_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 손 랜드마크 좌표 추출 및 제스처 인식
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

            thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
            pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

            thumb_pip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP]
            index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
            middle_pip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP]
            ring_pip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP]
            pinky_pip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP]

            # 손 영역 추출
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])

            # 좌표가 이미지 범위 내에 있는지 확인
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max)
            y_max = min(frame.shape[0], y_max)

            if x_max - x_min > 0 and y_max - y_min > 0:
                hand_rgb = rgb_frame[y_min:y_max, x_min:x_max]
                pil_hand_image = Image.fromarray(hand_rgb)

                # 손 영역의 깊이 맵 추정
                depth_map = depth_pipe(pil_hand_image)['depth']
                depth_map = np.array(depth_map)

                # 깊이 값을 반전하여 가까운 거리가 더 작게 나타나도록 조정
                depth_map = 1 / (depth_map + 1e-6)  # 0으로 나누는 것을 피하기 위해 작은 값 추가

                # 깊이 맵 시각화
                depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
                depth_colormap = cv2.resize(depth_colormap, (x_max - x_min, y_max - y_min))
                depth_colormap = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2BGR)

                # 크기 및 채널 일치 확인
                if depth_frame[y_min:y_max, x_min:x_max].shape == depth_colormap.shape:
                    # 깊이 맵을 원본 프레임에 합성
                    depth_frame[y_min:y_max, x_min:x_max] = cv2.addWeighted(
                        depth_frame[y_min:y_max, x_min:x_max], 0.7, depth_colormap, 0.3, 0
                    )

                # 손의 중심 깊이 값 계산
                center_depth = depth_map[depth_map.shape[0] // 2, depth_map.shape[1] // 2]
                distance = center_depth * 1000  # 깊이 값을 mm로 변환 (예제용으로 1000을 곱함)
                cv2.putText(depth_frame, f'Distance: {distance:.2f} mm', (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

            # 손가락 하트 (small heart) 제스처 인식
            thumb_tip_x = thumb_tip.x * frame.shape[1]
            thumb_tip_y = thumb_tip.y * frame.shape[0]
            index_tip_x = index_tip.x * frame.shape[1]
            index_tip_y = index_tip.y * frame.shape[0]

            # 엄지와 검지가 서로 가까이 위치
            distance_between_tips = np.sqrt((thumb_tip_x - index_tip_x) ** 2 + (thumb_tip_y - index_tip_y) ** 2)

            # 엄지와 검지가 교차하는지 확인
            if distance_between_tips < 30 and (thumb_tip_x < index_tip_x or thumb_tip_x > index_tip_x):
                cv2.putText(frame, 'Small Heart Gesture', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(depth_frame, 'Small Heart Gesture', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # 뻐큐 손가락 제스처 인식
            # 중지가 똑바로 세워져 있고 다른 손가락이 구부러진 상태 확인
            if (middle_tip.y < index_tip.y and middle_tip.y < ring_tip.y and 
                middle_tip.y < pinky_tip.y and middle_tip.y < thumb_tip.y and 
                middle_tip.y < middle_pip.y and
                index_tip.y > index_pip.y and ring_tip.y > ring_pip.y and 
                pinky_tip.y > pinky_pip.y and thumb_tip.y > thumb_pip.y):
                print("Detected middle finger gesture. Exiting...")
                cap.release()
                cv2.destroyAllWindows()
                exit()

    # 두 이미지를 나란히 결합
    combined_image = np.hstack((frame, depth_frame))
    
    # 결과를 화면에 표시
    cv2.imshow('Hand Gesture Recognition: RGB (Left) vs RGBD (Right)', combined_image)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\lkg\Desktop\vscode\torch\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase

### 손 포인트 상대거리

In [2]:
import cv2
import numpy as np
from transformers import pipeline
from PIL import Image
import torch
import mediapipe as mp

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
depth_pipe = pipeline("depth-estimation", device=device)

# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 손 랜드마크 검출
    results = hands.process(rgb_frame)

    depth_frame = frame.copy()  # 깊이 정보 포함한 프레임 복사본

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 랜드마크 그리기
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 손 영역 추출
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])

            # 좌표가 이미지 범위 내에 있는지 확인
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max)
            y_max = min(frame.shape[0], y_max)

            if x_max - x_min > 0 and y_max - y_min > 0:
                hand_rgb = rgb_frame[y_min:y_max, x_min:x_max]
                pil_hand_image = Image.fromarray(hand_rgb)

                # 손 영역의 깊이 맵 추정
                depth_result = depth_pipe(pil_hand_image)
                depth_map = np.array(depth_result['depth'])

                # 깊이 값을 반전하지 않음
                # 깊이 맵 시각화
                depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
                depth_colormap = cv2.resize(depth_colormap, (x_max - x_min, y_max - y_min))
                depth_colormap = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2BGR)

                # 크기 및 채널 일치 확인
                if depth_frame[y_min:y_max, x_min:x_max].shape == depth_colormap.shape:
                    # 깊이 맵을 원본 프레임에 합성
                    depth_frame[y_min:y_max, x_min:x_max] = cv2.addWeighted(
                        depth_frame[y_min:y_max, x_min:x_max], 0.7, depth_colormap, 0.3, 0
                    )

                # 손의 중심 깊이 값 계산
                center_depth = depth_map[depth_map.shape[0] // 2, depth_map.shape[1] // 2]

                # 각 손가락 마디에 대한 깊이 정보 점으로 표시
                for lm in mp_hands.HandLandmark:
                    landmark = hand_landmarks.landmark[lm]
                    lm_x = int(landmark.x * frame.shape[1])
                    lm_y = int(landmark.y * frame.shape[0])
                    if 0 <= lm_x < frame.shape[1] and 0 <= lm_y < frame.shape[0]:
                        depth_value = depth_map[int(landmark.y * depth_map.shape[0]), int(landmark.x * depth_map.shape[1])]
                        relative_depth = depth_value / center_depth
                        cv2.circle(depth_frame, (lm_x, lm_y), 5, (0, 255, 0), -1)
                        cv2.putText(depth_frame, f'{relative_depth:.2f}', (lm_x, lm_y - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

    # 두 이미지를 나란히 결합
    combined_image = np.hstack((frame, depth_frame))
    
    # 결과를 화면에 표시
    cv2.imshow('Hand Depth Information: RGB (Left) vs RGBD (Right)', combined_image)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\lkg\Desktop\vscode\torch\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase

In [9]:
import cv2
import numpy as np
from transformers import pipeline
from PIL import Image
import torch
import mediapipe as mp

# 깊이 추정 파이프라인 초기화
device = 0 if torch.cuda.is_available() else -1
depth_pipe = pipeline("depth-estimation", device=device)

# 객체 인식 모델 초기화 (YOLOv3 모델 사용)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# COCO 클래스 이름 불러오기
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# MediaPipe Hands 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # 원본 프레임
    original_frame = frame.copy()

    # 객체 인식
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    frame_objects = frame.copy()
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame_objects, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame_objects, label + " " + str(round(confidence, 2)), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)

    # 손 포인트 인식 및 깊이 추정
    frame_points = frame.copy()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame_points, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])

            x_min = max(0, x_min)
            y_min = max(0, y_min)
            x_max = min(frame.shape[1], x_max)
            y_max = min(frame.shape[0], y_max)

            if x_max - x_min > 0 and y_max - y_min > 0:
                hand_rgb = rgb_frame[y_min:y_max, x_min:x_max]
                pil_hand_image = Image.fromarray(hand_rgb)
                depth_result = depth_pipe(pil_hand_image)
                depth_map = np.array(depth_result['depth'])

                depth_colormap = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
                depth_colormap = cv2.applyColorMap(depth_colormap, cv2.COLORMAP_JET)
                depth_colormap = cv2.resize(depth_colormap, (x_max - x_min, y_max - y_min))
                depth_colormap = cv2.cvtColor(depth_colormap, cv2.COLOR_RGB2BGR)

                if frame_points[y_min:y_max, x_min:x_max].shape == depth_colormap.shape:
                    frame_points[y_min:y_max, x_min:x_max] = cv2.addWeighted(
                        frame_points[y_min:y_max, x_min:x_max], 0.7, depth_colormap, 0.3, 0
                    )

                center_depth = depth_map[depth_map.shape[0] // 2, depth_map.shape[1] // 2]
                for lm in mp_hands.HandLandmark:
                    landmark = hand_landmarks.landmark[lm]
                    lm_x = int(landmark.x * frame.shape[1])
                    lm_y = int(landmark.y * frame.shape[0])
                    if 0 <= lm_x < frame.shape[1] and 0 <= lm_y < frame.shape[0]:
                        depth_value = depth_map[int(landmark.y * depth_map.shape[0]), int(landmark.x * depth_map.shape[1])]
                        relative_depth = depth_value / center_depth
                        cv2.circle(frame_points, (lm_x, lm_y), 5, (0, 255, 0), -1)
                        cv2.putText(frame_points, f'{relative_depth:.2f}', (lm_x, lm_y - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

    # 1행 3열로 이미지 결합
    combined_frame = np.hstack((original_frame, frame_objects, frame_points))
    
    # 결과를 화면에 표시
    cv2.imshow('Combined View', combined_frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No model was supplied, defaulted to Intel/dpt-large and revision e93beec (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
